In [5]:
1+2

3

In [6]:
pip install "fastapi[all]"

  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached starlette-0.47.2-py3-none-any.whl.metadata (6.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 345.6 kB/s eta 0:00:00 0:00:01
  Using cached email_validator-2.2.0-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached email_validator-2.2.0-py3-none-any.whl (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.5/226.5 kB 5.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 7.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# --- Cell 1: Install Dependencies ---
!pip install pydantic
!pip install pydantic-settings
!pip install sqlmodel
!# --- Cell 1: Install dependencies ---
!pip install fastapi uvicorn sqlmodel requests python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [sqlmodel]2/3 [sqlmodel]y]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [requests]1/2 [requests]


In [8]:
from fastapi import FastAPI, APIRouter, Depends, Request, Header, HTTPException, status
from fastapi.responses import JSONResponse
from pydantic_settings import BaseSettings
from pydantic import BaseModel, Field, conint, confloat
from sqlmodel import SQLModel, Field as SQLField, create_engine, Session, select, Column

In [9]:
# --- Cell 2: Imports & Settings ---
from fastapi import FastAPI, APIRouter, Depends, Request, Header, HTTPException, status
from fastapi.responses import JSONResponse
from pydantic_settings import BaseSettings
from pydantic import BaseModel, Field, conint, confloat
from sqlmodel import SQLModel, Field as SQLField, create_engine, Session, select, Column
from sqlalchemy import String  # add this import at the top of the cell
from typing import Optional, List
from datetime import datetime
import enum, hmac, hashlib, time, json


# Settings
class Settings(BaseSettings):
    PROJECT_NAME: str = "Orders & Inventory API"
    VERSION: str = "0.1.0"
    DATABASE_URL: str = "sqlite:///./test.db"
    WEBHOOK_SECRET: str = "changeme"
    MAX_WEBHOOK_AGE_SECONDS: int = 300
    class Config:
        env_file = ".env"

settings = Settings()

In [10]:
# --- Cell 3: Database setup ---
engine = create_engine(settings.DATABASE_URL, connect_args={"check_same_thread": False})

def init_db():
    SQLModel.metadata.create_all(engine)

def get_session():
    with Session(engine) as session:
        yield session

In [11]:
# --- Cell 4: Models ---
class OrderStatus(str, enum.Enum):
    PENDING = "PENDING"
    PAID = "PAID"
    SHIPPED = "SHIPPED"
    CANCELED = "CANCELED"

class Product(SQLModel, table=True):
    id: Optional[int] = SQLField(default=None, primary_key=True)
    sku: str = SQLField(sa_column=Column(String, unique=True, index=True))
    name: str
    price: float
    stock: int

class Order(SQLModel, table=True):
    id: Optional[int] = SQLField(default=None, primary_key=True)
    product_id: int = SQLField(foreign_key="product.id")
    quantity: int
    status: OrderStatus = SQLField(default=OrderStatus.PENDING)
    created_at: datetime = SQLField(default_factory=datetime.utcnow)

class WebhookEvent(SQLModel, table=True):
    id: Optional[int] = SQLField(default=None, primary_key=True)
    event_id: str = SQLField(unique=True, index=True)
    created_at: datetime = SQLField(default_factory=datetime.utcnow)

In [12]:
# --- Cell 5: Schemas ---
class ProductCreate(BaseModel):
    sku: str
    name: str
    price: confloat(gt=0)
    stock: conint(ge=0)

class ProductRead(BaseModel):
    id: int
    sku: str
    name: str
    price: float
    stock: int
    class Config:
        orm_mode = True

class ProductUpdate(BaseModel):
    sku: Optional[str] = None
    name: Optional[str] = None
    price: Optional[confloat(gt=0)] = None
    stock: Optional[conint(ge=0)] = None

class OrderCreate(BaseModel):
    product_id: int
    quantity: conint(gt=0)

class OrderRead(BaseModel):
    id: int
    product_id: int
    quantity: int
    status: OrderStatus
    created_at: datetime
    class Config:
        orm_mode = True

class OrderUpdate(BaseModel):
    quantity: Optional[conint(gt=0)] = None
    status: Optional[OrderStatus] = None

class WebhookPayload(BaseModel):
    event: str
    order_id: int
    metadata: Optional[dict] = None

/Users/sameermeher/Technical/vscode/fastapi-order-inventory/myenv/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [13]:
# --- Cell 6: CRUD operations ---
def create_product(session: Session, data: ProductCreate) -> Product:
    if session.exec(select(Product).where(Product.sku == data.sku)).first():
        raise HTTPException(status_code=409, detail="SKU already exists")
    product = Product(**data.dict())
    session.add(product)
    session.commit()
    session.refresh(product)
    return product

def list_products(session: Session, offset=0, limit=100) -> List[Product]:
    return session.exec(select(Product).offset(offset).limit(limit)).all()

def get_product(session: Session, pid: int):
    return session.get(Product, pid)

def update_product(session: Session, pid: int, data: ProductUpdate) -> Product:
    product = get_product(session, pid)
    if not product:
        raise HTTPException(status_code=404, detail="Product not found")
    for k, v in data.dict(exclude_unset=True).items():
        setattr(product, k, v)
    session.add(product)
    session.commit()
    session.refresh(product)
    return product

def delete_product(session: Session, pid: int):
    product = get_product(session, pid)
    if not product:
        raise HTTPException(status_code=404, detail="Product not found")
    session.delete(product)
    session.commit()

def create_order(session: Session, data: OrderCreate) -> Order:
    product = get_product(session, data.product_id)
    if not product:
        raise HTTPException(status_code=404, detail="Product not found")
    if product.stock < data.quantity:
        raise HTTPException(status_code=409, detail="Insufficient stock")
    product.stock -= data.quantity
    order = Order(product_id=data.product_id, quantity=data.quantity)
    session.add(order)
    session.add(product)
    session.commit()
    session.refresh(order)
    return order

def get_order(session: Session, oid: int):
    return session.get(Order, oid)

def mark_order_paid(session: Session, oid: int) -> Order:
    order = get_order(session, oid)
    if not order:
        raise HTTPException(status_code=404, detail="Order not found")
    order.status = OrderStatus.PAID
    session.add(order)
    session.commit()
    session.refresh(order)
    return order

def save_webhook_event(session: Session, event_id: str):
    if session.exec(select(WebhookEvent).where(WebhookEvent.event_id == event_id)).first():
        raise HTTPException(status_code=409, detail="Event already processed")
    session.add(WebhookEvent(event_id=event_id))
    session.commit()

In [14]:
# --- Cell 7: Security helpers ---
def compute_signature(secret: str, timestamp: str, body: bytes) -> str:
    msg = timestamp.encode() + b"." + body
    return hmac.new(secret.encode(), msg, hashlib.sha256).hexdigest()

def verify_signature(sig: str, timestamp: str, body: bytes):
    if not sig or not timestamp:
        raise HTTPException(status_code=401, detail="Missing signature")
    try:
        ts_int = int(timestamp)
    except ValueError:
        raise HTTPException(status_code=401, detail="Invalid timestamp")
    if abs(int(time.time()) - ts_int) > settings.MAX_WEBHOOK_AGE_SECONDS:
        raise HTTPException(status_code=401, detail="Signature too old")
    expected = compute_signature(settings.WEBHOOK_SECRET, timestamp, body)
    if not hmac.compare_digest(expected, sig):
        raise HTTPException(status_code=401, detail="Invalid signature")

In [15]:
# --- Cell 8: Routers ---
products_router = APIRouter(prefix="/products", tags=["products"])
orders_router = APIRouter(prefix="/orders", tags=["orders"])
webhooks_router = APIRouter(prefix="/webhooks", tags=["webhooks"])

@products_router.post("", response_model=ProductRead, status_code=201)
def create_product_ep(payload: ProductCreate, session: Session = Depends(get_session)):
    return create_product(session, payload)

@products_router.get("", response_model=List[ProductRead])
def list_products_ep(session: Session = Depends(get_session)):
    return list_products(session)

@orders_router.post("", response_model=OrderRead, status_code=201)
def create_order_ep(payload: OrderCreate, session: Session = Depends(get_session)):
    return create_order(session, payload)

@webhooks_router.post("/payment")
async def payment_webhook(request: Request,
                          x_signature: str = Header(None, alias="X-Signature"),
                          x_timestamp: str = Header(None, alias="X-Signature-Timestamp"),
                          x_event_id: str = Header(None, alias="X-Event-Id"),
                          session: Session = Depends(get_session)):
    body = await request.body()
    verify_signature(x_signature, x_timestamp, body)
    if not x_event_id:
        raise HTTPException(status_code=400, detail="Missing X-Event-Id")
    save_webhook_event(session, x_event_id)
    payload = WebhookPayload.parse_raw(body)
    if payload.event == "payment.succeeded":
        mark_order_paid(session, payload.order_id)
        return {"status": "ok", "detail": f"Order {payload.order_id} marked PAID"}
    return {"status": "ignored"}
    

In [16]:
# --- Cell 9: App factory ---
app = FastAPI(title=settings.PROJECT_NAME, version=settings.VERSION)

@app.on_event("startup")
def startup():
    init_db()

app.include_router(products_router)
app.include_router(orders_router)
app.include_router(webhooks_router)

/var/folders/9f/rq1t2qzd2bs8kms6ky16kfjr0000gn/T/ipykernel_8367/20291762.py:4: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [17]:
# --- Cell 10: Run Uvicorn inside notebook ---
import uvicorn
from threading import Thread

def run_app():
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info")

server_thread = Thread(target=run_app, daemon=True)
server_thread.start()

INFO:     Started server process [8367]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50177 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50177 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:50177 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50177 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50178 - "GET /products HTTP/1.1" 200 OK
INFO:     127.0.0.1:50184 - "POST /products HTTP/1.1" 201 Created


/var/folders/9f/rq1t2qzd2bs8kms6ky16kfjr0000gn/T/ipykernel_8367/4287500694.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  product = Product(**data.dict())


INFO:     127.0.0.1:50185 - "GET /products HTTP/1.1" 200 OK
INFO:     127.0.0.1:50187 - "POST /products HTTP/1.1" 201 Created
INFO:     127.0.0.1:50188 - "POST /products HTTP/1.1" 409 Conflict
INFO:     127.0.0.1:50189 - "GET /products HTTP/1.1" 200 OK
INFO:     127.0.0.1:50190 - "POST /orders HTTP/1.1" 201 Created
INFO:     127.0.0.1:50191 - "GET /products HTTP/1.1" 200 OK
INFO:     127.0.0.1:50311 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50311 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50312 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50312 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50312 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50312 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50312 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50312 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50327 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50327 - "GET /openapi.json HTTP/1.1" 200 OK


In [21]:
server.should_exit = True

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGTERM)


: 